### Cell 1: Imports
This cell contains all the necessary libraries. We're adding RandomForestClassifier and pickle for saving/loading the layer outputs.

In [ ]:
import json
import os
import pandas as pd
import numpy as np
import time
import subprocess
import gc
import pickle  # Added for saving/loading layer outputs
import tensorflow as tf
import cv2 # OpenCV for robust image loading/resizing
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import layers, models, backend as K # ⬅️ THIS LINE IS FIXED
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier # ⬅️ NEW: Import Random Forest
from tensorflow.keras.models import Model # ⬅️ NEW: For creating feature extractors

### Configuration
This cell holds all your global constants. This makes it easy to tweak your experiment (e.g., change IMG_HEIGHT, BATCH_SIZE, or N_FOLDS) without digging through the code.

In [ ]:
# --- Configuration ---
BASE_DATA_DIR       = "../00_download_dataset/dogs-vs-cats"
TRAIN_DATA_DIR      = os.path.join(BASE_DATA_DIR, "train")
CSV_METADATA_FILE   = os.path.join(BASE_DATA_DIR, "train_metadata.csv")
RESULTS_DIR         = "./results" # ⬅️ NEW: A dedicated folder for results
RESULTS_CSV_PATH    = os.path.join(RESULTS_DIR, "hybrid_cnn_rf_kfold_results.csv")
ACTIVATION_PKL_PATH = os.path.join(RESULTS_DIR, "last_fold_activations.pkl")


IMG_HEIGHT          = 128 # 128 # 250
IMG_WIDTH           = 128 # 250
# CRITICAL: Batch size of 12 is good for 250x250 images on a single GPU
BATCH_SIZE          = 12 
RANDOM_STATE        = 42
MAX_SAMPLES_PER_CLASS = 5000 # 5000 # 22000
NUM_CLASSES         = 2 
N_FOLDS             = 5
EPOCHS              = 20 # ⬅️ NEW: Define epochs here for clarity

### GPU & Memory Management
These are your helper functions to manage system resources, especially for TensorFlow. Running setup_tensorflow_memory() early is good practice.

In [ ]:
def list_available_gpus():
    # ... (Your original code)
    physical_gpus = tf.config.list_physical_devices('GPU')
    print(f"Num GPUs Available: {len(physical_gpus)}")
    for gpu in physical_gpus:
        print(f"  {gpu}")

def setup_tensorflow_memory(force_cpu=False):
    """
    MODIFIED: This function now targets a SINGLE GPU (GPU:1)
    to simplify resource management.
    """
    physical_gpus = tf.config.list_physical_devices('GPU')
    
    if force_cpu:
        print("🖥️ Forcing CPU-only mode...")
        tf.config.set_visible_devices([], 'GPU')
        print("✅ CPU-only mode enabled")
        return

    # ⬇️ --- THIS IS THE MODIFICATION --- ⬇️
    # We will target *only* GPU:1.
    # Change the index [1] if you want to use GPU:2 or GPU:3 instead.
    TARGET_GPU_INDEX = 1 
    
    if physical_gpus and len(physical_gpus) > TARGET_GPU_INDEX:
        
        # Select ONLY the target GPU
        gpus_to_configure = [physical_gpus[TARGET_GPU_INDEX]]
        
        # Tell TensorFlow to make *only* this GPU visible
        tf.config.set_visible_devices(gpus_to_configure, 'GPU')
        
        print(f"🎯 Configuring ONLY Physical GPU {TARGET_GPU_INDEX}.")
    # ⬆️ --- END OF MODIFICATION --- ⬆️
    
    else:
        # Fallback in case GPU:1 isn't found
        gpus_to_configure = physical_gpus
        print(f"⚠️ Could not find GPU {TARGET_GPU_INDEX}. Configuring all available GPUs.")

    if not gpus_to_configure:
        print("❌ No GPUs detected. Using CPU.")
        return

    try:
        # Apply memory growth to our selected GPU
        for gpu in gpus_to_configure:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        tf.config.optimizer.set_jit(True) 
        print("⚡ XLA optimization enabled.")
        
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"✅ Successfully configured {len(logical_gpus)} logical GPU(s).")
        
    except RuntimeError as e:
        print(f"❌ GPU configuration error: {e}")

def aggressive_memory_cleanup():
    tf.keras.backend.clear_session()
    gc.collect()
    time.sleep(0.5)
    print("🧹 Aggressive memory cleanup completed")
    

# --- Setup Memory ---
# Create results dir
os.makedirs(RESULTS_DIR, exist_ok=True)

# Run the NEW GPU setup
setup_tensorflow_memory()
list_available_gpus()

### Data Loading Helpers
These functions are responsible for loading your entire dataset from disk into two large NumPy arrays in RAM. This "in-memory" approach is fast for training after the initial load, but requires a lot of RAM.

In [ ]:
def prepare_metadata():
    """Reads or creates the metadata CSV and returns the filtered DataFrame."""
    if not os.path.exists(CSV_METADATA_FILE):
        # ... (Your original 2.1 CSV creation logic must be here) ...
        print(f"Error: {CSV_METADATA_FILE} not found!")
        print("Please run your dataset download/prep script first.")
        return None, None
         
    df = pd.read_csv(CSV_METADATA_FILE)

    # Apply sample limit (Original 2.2 logic)
    if MAX_SAMPLES_PER_CLASS is not None:
        df = df.groupby('class').apply(
            lambda x: x.sample(min(len(x), MAX_SAMPLES_PER_CLASS), random_state=RANDOM_STATE)
        ).reset_index(drop=True)

    # Create label mapping
    labels_text_all = df["class"].values
    unique_labels_sorted = sorted(list(set(labels_text_all)))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels_sorted)}
    index_to_label = {idx: label for label, idx in label_to_index.items()} # ⬅️ NEW: Good for plots
    
    df['label_numeric'] = df['class'].map(label_to_index)
    
    print(f"Metadata prepared: {len(df)} samples.")
    print(f"Label map: {label_to_index}")
    
    return df, label_to_index, index_to_label


def dataloader_catdog_numpy(df):
    """
    Loads all Cats vs. Dogs images into RAM, resizing and normalizing them.
    """
    print(f"⏱️ Starting image load/resize for {len(df)} samples into RAM...")
    x, y = [], []
    
    for index, row in df.iterrows():
        try:
            img = cv2.imread(row['path']) 
            if img is None:
                raise Exception(f"Could not read image {row['path']}")
                
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
            img = img.astype(np.float32) / 255.0      # Normalize
            
            x.append(img)
            y.append(row['label_numeric'])
            
        except Exception as e:
            print(f"Warning: {e}. Skipping.")
            continue
            
    print(f"✅ Data loaded into RAM: {len(x)} samples.")
    return np.array(x), np.array(y)

### Plotting & Activation Helpers
These are your utility functions.

The `plot_*` functions are for evaluating model performance at the end of a fold.

`get_all_layer_outputs` is your function to grab activations from all layers, which we'll use for visualization.

In [ ]:
def plot_training_history(history, fold_num):
    """Plots the training loss and accuracy over epochs."""
    print(f"\n📈 Plotting Training History for Fold {fold_num}...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot Loss
    ax1.plot(history.history['loss'], color='b', label="Training Loss")
    ax1.set_title(f"Fold {fold_num} - Training Loss")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.legend()
    
    # Plot Accuracy
    ax2.plot(history.history['accuracy'], color='g', label="Training Accuracy")
    ax2.set_title(f"Fold {fold_num} - Training Accuracy")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy")
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(y_true, y_pred, labels, title="Confusion Matrix"):
    """Generates and plots a Confusion Matrix."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt="d", 
        cmap="Blues", 
        xticklabels=labels, 
        yticklabels=labels
    )
    plt.title(title)
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

def plot_roc_curve(y_true, y_proba):
    """Calculates and plots the Receiver Operating Characteristic (ROC) curve."""
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(7, 7))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

def get_all_layer_outputs(model, x_data, batch_size=32):
    """
    Extracts and returns the outputs from every layer in a Keras model.
    Used for visualization.
    Returns a dict mapping layer names to their activation tensors (numpy arrays).
    """
    print(f"Extracting all layer outputs for {len(x_data)} samples...")
    
    # Collect all layer outputs (skip the Input layer)
    layer_outputs = [layer.output for layer in model.layers if 'input' not in layer.name.lower()]
    layer_names = [layer.name for layer in model.layers if 'input' not in layer.name.lower()]
    
    # Create a new model that returns all those outputs
    activation_model = Model(inputs=model.input, outputs=layer_outputs)
    
    # Run inference on the input data
    activations = activation_model.predict(x_data, batch_size=batch_size, verbose=1)
    
    # Handle single-layer case
    if len(layer_names) == 1:
        activations = [activations]
        
    # Package into a dictionary: {layer_name: activations}
    layer_output_dict = {}
    for name, act in zip(layer_names, activations):
        layer_output_dict[name] = act
        print(f"  -> Extracted {name:20s} | Shape: {act.shape}")
    
    return layer_output_dict

### CNN & Hybrid Model Helpers
This is where we define the models.

1. `build_cnn_model`: Your original function to create the full LeNet.

2. `create_feature_extractor`: (NEW) A crucial helper that takes a trained CNN and a layer name, then "chops off" the top and returns a new model that just outputs that layer's features.

3. `run_hybrid_rf_experiment`: (NEW) This function takes the data and one of the new extractor models, generates the features, flattens them, and runs the full RF train/test experiment.

In [ ]:
def build_cnn_model(input_shape):
    """
    Builds the LeNet-style CNN model using the stable Functional API approach.
    """
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    
    # Layer C1: Conv(6) -> Pool(2x2)
    # We add 'name' so we can easily target it later
    x = layers.Conv2D(6, (5, 5), activation='relu', name='conv_1')(x) 
    x = layers.MaxPooling2D(pool_size=(2, 2), name='pool_1')(x)
    
    # Layer C2: Conv(16) -> Pool(2x2)
    x = layers.Conv2D(16, (5, 5), activation='relu', name='conv_2')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name='pool_2')(x)
    
    # Flatten: Feature Vector
    x = layers.Flatten(name='flatten')(x)
    
    # Dense Layers (C5, F6)
    x = layers.Dense(120, activation='relu', name='dense_1')(x)
    x = layers.Dense(84, activation='relu', name='dense_2')(x)
    
    # Output Layer
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

def create_feature_extractor(full_trained_model, layer_name):
    """
    Takes a fully trained Keras model and the name of an intermediate
    layer, and returns a NEW model that outputs that layer's activations.
    """
    try:
        # Create a new model that shares the same inputs as the
        # original, but outputs the activations from the desired layer.
        extractor_model = Model(
            inputs=full_trained_model.input,
            outputs=full_trained_model.get_layer(layer_name).output
        )
        return extractor_model
    except ValueError as e:
        print(f"Error creating extractor for layer '{layer_name}': {e}")
        print("Check your model's layer names with model.summary()")
        return None

def run_hybrid_rf_experiment(extractor_model, x_train, y_train, x_test, y_test, batch_size):
    """
    Uses a feature_extractor model to generate features, then
    trains and evaluates a Random Forest on them.
    """
    layer_name = extractor_model.output.name.split('/')[0] # Get clean layer name
    print(f"--- 🌳 Starting Hybrid RF Experiment for layer: {layer_name} ---")
    
    # 1. Generate features
    print(f"Generating features for {x_train.shape[0]} train samples...")
    start_feat_time = time.time()
    x_train_features = extractor_model.predict(x_train, batch_size=batch_size, verbose=1)
    print(f"Generating features for {x_test.shape[0]} test samples...")
    x_test_features = extractor_model.predict(x_test, batch_size=batch_size, verbose=1)
    feat_gen_time = time.time() - start_feat_time
    print(f"Feature generation took {feat_gen_time:.2f}s")

    # 2. Flatten features
    # RF needs 2D input: (n_samples, n_features)
    # Conv layers output 4D: (n_samples, H, W, C)
    print(f"Original feature shape: {x_train_features.shape}")
    if x_train_features.ndim > 2:
        # Reshape (e.g., 35200, 59, 59, 16) -> (35200, 59*59*16)
        n_samples_train = x_train_features.shape[0]
        n_samples_test = x_test_features.shape[0]
        
        x_train_features_flat = x_train_features.reshape(n_samples_train, -1)
        x_test_features_flat = x_test_features.reshape(n_samples_test, -1)
    else:
        x_train_features_flat = x_train_features
        x_test_features_flat = x_test_features
        
    print(f"Flattened feature shape: {x_train_features_flat.shape}")

    # 3. Train Random Forest
    print("Training Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=100, # A good, fast default
        random_state=RANDOM_STATE, 
        n_jobs=-1 # Use all available CPU cores
    )
    start_rf_train = time.time()
    rf_model.fit(x_train_features_flat, y_train)
    rf_train_time = time.time() - start_rf_train
    print(f"RF training took {rf_train_time:.2f}s")
    
    # 4. Evaluate Random Forest
    y_pred_rf = rf_model.predict(x_test_features_flat)
    
    # 5. Calculate metrics
    metrics = {
        'model_name': f'RF_from_{layer_name}',
        'accuracy': accuracy_score(y_test, y_pred_rf),
        'precision': precision_score(y_test, y_pred_rf, average='binary', zero_division=0),
        'recall': recall_score(y_test, y_pred_rf, average='binary', zero_division=0),
        'f1_score': f1_score(y_test, y_pred_rf, average='binary', zero_division=0),
        'train_time': rf_train_time,
        'feature_gen_time': feat_gen_time
    }
    
    print(f"✅ RF from {layer_name} Accuracy: {metrics['accuracy']:.4f}")
    return metrics

### Main K-Fold Runner (The "Orchestrator")
This is the most important cell. It's your original `standard_cnn_runner`, but heavily modified to run the new hybrid experiments inside the K-Fold loop.

Read the comments to see the flow:

1. Train Baseline CNN (Model A).

2. Get Baseline CNN metrics.

3. Define which layers to extract from.

4. Loop through each layer, create an extractor, and run the RF experiment (Model B, C, ...).

In [ ]:
def standard_cnn_rf_runner(df, label_map, n=N_FOLDS, epochs=EPOCHS, batch_size=BATCH_SIZE):
    """
    MODIFIED: Runs K-Fold Cross-Validation for BOTH the standard
    CNN and the hybrid CNN-RF models.
    """
    print("🚀 Starting Hybrid CNN-RF K-Fold Runner...")
    
    # Load all data into NumPy arrays
    x, y = dataloader_catdog_numpy(df)
    
    print(f"📈 Total samples loaded: {x.shape[0]}. Input shape: {x.shape[1:]}")
    input_shape = x.shape[1:]
    
    # This is the list of layers you want to test
    # We use the names we defined in build_cnn_model
    # We test every major feature-processing step
    LAYERS_TO_EXTRACT_FROM = [
        'conv_1',  # After 1st Conv
        'pool_1',  # After 1st Pool
        'conv_2',  # After 2nd Conv
        'pool_2',  # After 2nd Pool
        'flatten', # After Flatten
        'dense_1', # After 1st Dense
        'dense_2'  # After 2nd Dense
    ]
    
    all_results_list = []
    skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=RANDOM_STATE)

    for fold, (train_index, test_index) in enumerate(skf.split(x, y)):
        print(f"\n{'='*70}")
        print(f"🔄 Starting Fold {fold + 1}/{n}")
        print(f"{'='*70}")
        
        aggressive_memory_cleanup()
        
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        print(f"📊 Fold {fold + 1} - Train: {x_train.shape[0]}, Test: {x_test.shape[0]}")
        
        # =======================================================
        # 1. TRAIN BASELINE CNN (MODEL A)
        # =======================================================
        print("\n--- 🧠 Training Baseline CNN (Model A) ---")
        start_cnn_train = time.time()
        
        # Build and compile the model
        baseline_model = build_cnn_model(input_shape)
        baseline_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
            loss='binary_crossentropy', 
            metrics=['accuracy']
        )
        
        # Print a summary on the first fold to see layer names
        if fold == 0:
            print(baseline_model.summary())

        # Fit the model
        history = baseline_model.fit(
            x_train, y_train, 
            epochs=epochs, 
            batch_size=batch_size, 
            verbose=1,
            validation_data=None # We use the test set for final eval
        )
        
        cnn_train_time = time.time() - start_cnn_train
        print(f"⏱️ CNN training completed in {cnn_train_time:.2f} seconds")

        # Evaluate the CNN
        y_pred_cnn_proba = baseline_model.predict(x_test, batch_size=batch_size, verbose=0).flatten()
        y_pred_cnn_binary = (y_pred_cnn_proba > 0.5).astype(int)

        # Store CNN metrics
        cnn_metrics = {
            'model_name': 'Full_CNN (Baseline)',
            'run': fold + 1,
            'accuracy': accuracy_score(y_test, y_pred_cnn_binary),
            'precision': precision_score(y_test, y_pred_cnn_binary, average='binary', zero_division=0),
            'recall': recall_score(y_test, y_pred_cnn_binary, average='binary', zero_division=0),
            'f1_score': f1_score(y_test, y_pred_cnn_binary, average='binary', zero_division=0),
            'train_time': cnn_train_time
        }
        all_results_list.append(cnn_metrics)
        print(f"✅ Baseline CNN Accuracy: {cnn_metrics['accuracy']:.4f}")
        
        
        # =======================================================
        # 2. RUN HYBRID RF EXPERIMENTS (MODELS B, C, ...)
        # =======================================================
        print(f"\n--- 🌳 Starting Hybrid RF Experiments (Models B, C, ...) ---")
        
        for layer_name in LAYERS_TO_EXTRACT_FROM:
            aggressive_memory_cleanup()
            
            # Create a new extractor model from the *trained* baseline model
            extractor = create_feature_extractor(baseline_model, layer_name)
            
            if extractor:
                # Run the full RF experiment
                rf_metrics = run_hybrid_rf_experiment(
                    extractor_model=extractor,
                    x_train=x_train,
                    y_train=y_train,
                    x_test=x_test,
                    y_test=y_test,
                    batch_size=batch_size
                )
                rf_metrics['run'] = fold + 1
                all_results_list.append(rf_metrics)

        
        # =======================================================
        # 3. PLOTTING & VISUALIZATION (FOR LAST FOLD ONLY)
        # =======================================================
        if fold == n - 1:
            print(f"\n--- 📊 Final Fold Visualizations ---")
            
            # 1. Plot CNN Training History
            plot_training_history(history, fold + 1)
            
            # 2. Plot CNN Confusion Matrix
            target_names = [label_map[i] for i in range(len(label_map))]
            plot_confusion_matrix(y_test, y_pred_cnn_binary, target_names, 
                                  title=f"Baseline CNN Confusion Matrix (Fold {fold + 1})")
            
            # 3. Plot CNN ROC Curve
            plot_roc_curve(y_test, y_pred_cnn_proba)
            
            # 4. Save all layer activations AND inputs for ONE BATCH of test images
            # This is for your interpretability goal
            print("\nSaving layer activations and inputs for visualization...")
            test_subset = x_test[:batch_size]
            activation_dict = get_all_layer_outputs(baseline_model, test_subset, batch_size)
            
            # ⬇️ --- THIS IS THE MODIFICATION --- ⬇️
            data_to_save = {
                'inputs': test_subset,
                'activations': activation_dict
            }
            with open(ACTIVATION_PKL_PATH, "wb") as f:
                pickle.dump(data_to_save, f)
            # ⬆️ --- END OF MODIFICATION --- ⬆️
            
            print(f"💾 Saved inputs and activations to {ACTIVATION_PKL_PATH}")

        # Clean up the baseline model to free memory
        del baseline_model
        del x_train, x_test, y_train, y_test
        aggressive_memory_cleanup()
        print(f"🧹 Fold {fold + 1} cleanup completed")

    return all_results_list


def main_pipeline(results_df_path=RESULTS_CSV_PATH):
    """
    Main pipeline execution function.
    """
    # 1. Get Metadata
    df_metadata, label_to_index, index_to_label = prepare_metadata()
    if df_metadata is None:
        return
    
    # 2. Run the K-Fold Runner
    all_results = standard_cnn_rf_runner(
        df=df_metadata,
        label_map=index_to_label,
        n=N_FOLDS,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE
    )

    # 3. Save all results
    if all_results:
        results_df = pd.DataFrame(all_results)
        results_df.to_csv(results_df_path, index=False)
        print(f"\n{'='*70}")
        print(f"💾🏆 All experiment results saved to {results_df_path}")
        print(f"{'='*70}")
    
    return results_df

### 🚀 Run the Full Experiment
This is the cell you'll execute to run the entire K-Fold cross-validation process. It will take a long time!

Warning: This will train `1` (CNN) + `7` (RF models) = 8 models per fold. For 5 folds, this is 40 total models trained.

In [ ]:
# --- RUN THE EXPERIMENT ---

print("Starting the full K-Fold pipeline...")
print(f"This will run {N_FOLDS} folds.")
print(f"Each fold trains 1 CNN ({EPOCHS} epochs) and {7} Random Forest models.")
print("This may take a significant amount of time.")

start_pipeline = time.time()

final_results_df = main_pipeline(results_df_path=RESULTS_CSV_PATH)

end_pipeline = time.time()
print(f"✅✅✅ Full pipeline finished in {(end_pipeline - start_pipeline) / 60:.2f} minutes.")

# Display the head of the results
if not final_results_df.empty:
    print(final_results_df.head())

### 📊 Analyze the Results
After the run is complete, this cell loads the saved CSV and calculates the mean performance for each model type. This is the **final answer** to your research question.

In [ ]:
# --- Analyze the Results ---

try:
    # Load the results from the CSV
    results_df = pd.read_csv(RESULTS_CSV_PATH)
    
    print("📈 K-Fold Results (Mean Performance):")
    
    # We group by the 'model_name' and calculate the mean of all numeric metrics
    mean_results = results_df.groupby('model_name').mean(numeric_only=True)
    
    # Tidy up the output
    columns_to_show = ['accuracy', 'f1_score', 'precision', 'recall', 'train_time']
    mean_results = mean_results[columns_to_show].sort_values(by='accuracy', ascending=False)
    
    print(mean_results.round(4))
    
    # Plot the main comparison
    mean_results['accuracy'].plot(
        kind='bar', 
        figsize=(12, 6), 
        title='Mean Accuracy Comparison (Higher is Better)',
        ylabel='Mean Accuracy'
    )
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
    
    # Plot training time
    mean_results['train_time'].plot(
        kind='bar', 
        figsize=(12, 6), 
        title='Mean Training Time Comparison (Lower is Better)',
        ylabel='Mean Train Time (seconds)',
        color='orange'
    )
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
    
except FileNotFoundError:
    print(f"Results file not found. Did you run the experiment in Cell 8?")
except Exception as e:
    print(f"An error occurred during analysis: {e}")

### 🖼️ Visualization Helpers
These are the functions to visualize the activations you saved in the `.pkl` file.

* `visualize_input_image`: Shows the original input image.
* `visualize_activation_maps_for_sample`: Shows feature maps for Conv/Pool layers for one image.
* `visualize_dense_layer`: Shows the 1D activation vector for a Dense layer for one image.
* `visualize_average_feature_maps`: Shows the *average* feature maps across the whole batch.

In [ ]:
def visualize_input_image(loaded_data, image_index=0):
    """
    Displays the original input image from the saved data.
    """
    try:
        # data['inputs'] has shape (batch_size, H, W, C)
        img = loaded_data['inputs'][image_index]
        
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title(f"Input Image (Sample {image_index})")
        plt.axis('off')
        plt.show()
        
    except Exception as e:
        print(f"Error plotting input image: {e}")

def visualize_activation_maps_for_sample(loaded_data, layer_name, image_index=0, max_features=16):
    """
    Visualize the first few feature maps from a convolution or pooling layer
    for a SINGLE image.
    """
    print(f"Visualizing {layer_name} for image index {image_index}")
    
    layer_outputs = loaded_data['activations']
    if layer_name not in layer_outputs:
        print(f"Error: Layer '{layer_name}' not found. Available: {list(layer_outputs.keys())}")
        return

    # shape: (H, W, channels)
    feature_maps = layer_outputs[layer_name][image_index] 
    
    if feature_maps.ndim != 3:
        print(f"Error: Layer {layer_name} is not a 3D (H, W, C) tensor. Skipping.")
        return
        
    num_features = min(feature_maps.shape[-1], max_features)
    
    cols = 4
    rows = int(np.ceil(num_features / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
    axes = axes.flatten()
    
    for i in range(num_features):
        axes[i].imshow(feature_maps[:, :, i], cmap='viridis')
        axes[i].set_title(f"Filter {i+1}")
        axes[i].axis('off')
        
    for j in range(num_features, len(axes)):
        axes[j].axis('off')
        
    plt.suptitle(f"Activations for {layer_name} (Sample {image_index})", fontsize=16)
    plt.tight_layout()
    plt.show()

def visualize_dense_layer(loaded_data, layer_name, image_index=0):
    """
    Visualizes the activations of a 1D (Dense) layer as a heatmap.
    """
    print(f"Visualizing {layer_name} for image index {image_index}")
    
    layer_outputs = loaded_data['activations']
    if layer_name not in layer_outputs:
        print(f"Error: Layer '{layer_name}' not found. Available: {list(layer_outputs.keys())}")
        return

    # shape: (1, n_neurons) or (n_neurons,)
    activations = layer_outputs[layer_name][image_index]
    
    # Ensure it's 2D for the heatmap
    if activations.ndim == 1:
        activations = np.expand_dims(activations, axis=0) # -> (1, n_neurons)
        
    if activations.ndim > 2 or (activations.shape[0] != 1 and activations.shape[1] != 1):
        print(f"Error: Layer {layer_name} is not a 1D vector. Skipping heatmap.")
        return

    plt.figure(figsize=(15, 3))
    sns.heatmap(activations, annot=True, fmt='.2f', cmap='viridis', cbar=False)
    plt.title(f"Activations for {layer_name} (Sample {image_index})")
    plt.xlabel("Neuron Index")
    plt.ylabel("Activation")
    plt.yticks([])
    plt.show()


def visualize_average_feature_maps(loaded_data, layer_name, max_features=16):
    """
    Calculates and visualizes the AVERAGE feature map across all samples
    in the batch for a specific layer.
    """
    print(f"Visualizing AVERAGE {layer_name} across batch")
    
    layer_outputs = loaded_data['activations']
    if layer_name not in layer_outputs:
        print(f"Error: Layer '{layer_name}' not found. Available: {list(layer_outputs.keys())}")
        return

    # activations shape: (batch_size, H, W, channels)
    activations = layer_outputs[layer_name]
    
    if activations.ndim != 4:
        print(f"Error: Layer {layer_name} is not a 4D (N, H, W, C) tensor. Skipping.")
        return
    
    # Calculate the mean across the batch (axis=0)
    mean_activations = np.mean(activations, axis=0) # shape: (H, W, channels)
    
    num_features = min(mean_activations.shape[-1], max_features)
    
    cols = 4
    rows = int(np.ceil(num_features / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
    axes = axes.flatten()
    
    for i in range(num_features):
        axes[i].imshow(mean_activations[:, :, i], cmap='viridis')
        axes[i].set_title(f"Filter {i+1} (Average)")
        axes[i].axis('off')
        
    for j in range(num_features, len(axes)):
        axes[j].axis('off')
        
    plt.suptitle(f"Average Activations for {layer_name} (Across Batch)", fontsize=16)
    plt.tight_layout()
    plt.show()

### 👁️ Run Visualizations

Now that the experiment is complete, let's load the `.pkl` file saved from the *last* fold. This file contains a batch of input images and all the corresponding layer activations from the trained baseline CNN.

We will step through the model, from input to output, to build an intuition for what it learned.

In [ ]:
# --- Load the Saved Data ---
try:
    with open(ACTIVATION_PKL_PATH, "rb") as f:
        loaded_data = pickle.load(f)
    
    print(f"✅ Loaded activations from {ACTIVATION_PKL_PATH}")
    print(f"Available layers: {list(loaded_data['activations'].keys())}")
    
    # We will analyze the first image in the batch
    IMG_TO_ANALYZE = 0

except FileNotFoundError:
    print(f"❌ Activation file not found. Did you run the experiment in Cell 8?")
except Exception as e:
    print(f"An error occurred during loading: {e}")

#### Visualization 1: The Input Image

First, let's look at the original image we will be tracing through the network. This gives us context for all the activation maps that follow.

In [ ]:
visualize_input_image(loaded_data, image_index=IMG_TO_ANALYZE)

#### Visualization 2: Early Layer (`conv_1`)

This is the output right after the **first** convolutional layer. We expect these filters to learn very simple, low-level features like edges, corners, and basic textures.

* **Observe:** Do some filters activate on horizontal lines? Vertical lines? Bright spots? Dark spots? This is the model's first "look" at the image.

In [ ]:
visualize_activation_maps_for_sample(loaded_data, 'conv_1', image_index=IMG_TO_ANALYZE)

#### Visualization 3: Deeper Layer (`conv_2`)

This is the output after the **second** convolutional layer. These filters have a larger receptive field (they "see" more of the original image) and combine the simple features from `conv_1` into more complex patterns.

* **Observe:** Are these patterns more complex? Can you start to see filters that respond to "furry" textures, "eye-like" shapes, or "ear-like" curves? The features are becoming more abstract.

In [ ]:
visualize_activation_maps_for_sample(loaded_data, 'conv_2', image_index=IMG_TO_ANALYZE)

#### Visualization 4: Dense Layer (`dense_1`)

After `conv_2`, the features are flattened into a long 1D vector and fed into the first Dense layer. This layer (`dense_1`) has 120 neurons.

We can't view this as a 2D image, but we can use a **heatmap** to see which of the 120 neurons are firing (have a high activation value).

* **Observe:** This vector is the "feature representation" that your Random Forest models `RF_from_dense_1` and `RF_from_dense_2` are using. A sparse activation pattern (mostly low values) is common.

In [ ]:
visualize_dense_layer(loaded_data, 'dense_1', image_index=IMG_TO_ANALYZE)

#### Visualization 5: Average Activations (What the Model *Generally* Looks For)

The visualizations above were for a *single image*. What does the model look for *in general*?

By averaging the activation maps across the entire batch, we can get a sense of the "prototypical" feature each filter is searching for.

* **Observe:** The "average" feature maps might look blurrier, but they highlight the *most common* patterns that trigger each filter. This helps confirm if `conv_1` is truly a general edge detector and `conv_2` is a general pattern/texture detector.

In [ ]:
print("--- 🧐 Displaying Average Activations for conv_1 ---")
visualize_average_feature_maps(loaded_data, 'conv_1')

print("\n--- 🧐 Displaying Average Activations for conv_2 ---")
visualize_average_feature_maps(loaded_data, 'conv_2')

### 💾 Save Average Activations

This cell fulfills the goal of saving the averaged tensor outputs. It loads the full activation pickle file, iterates through each layer, computes the mean activation across the batch, and saves these mean tensors into a new, smaller file.

This file is ideal for future analysis or graphing, as it represents what the model *generally* "sees" at each layer for this batch.

In [ ]:
import numpy as np
import pickle
import os

# Define the new path for the averaged activations
AVG_ACTIVATION_PKL_PATH = os.path.join(RESULTS_DIR, "last_fold_average_activations.pkl")

try:
    # Load the full data from the main run
    with open(ACTIVATION_PKL_PATH, "rb") as f:
        loaded_data = pickle.load(f)
    
    all_activations = loaded_data['activations']
    average_activations = {}
    
    print(f"Calculating average activations...")
    
    # Loop through each layer's activations
    for layer_name, activations in all_activations.items():
        # activations shape is (batch_size, ...)
        # We calculate the mean along the batch axis (axis=0)
        try:
            mean_act = np.mean(activations, axis=0)
            average_activations[layer_name] = mean_act
            print(f"  -> Calculated mean for {layer_name:20s} | New Shape: {mean_act.shape}")
        except Exception as e:
            print(f"  -> Error calculating mean for {layer_name}: {e}")
            
    # Save the new dictionary of mean activations
    with open(AVG_ACTIVATION_PKL_PATH, "wb") as f:
        pickle.dump(average_activations, f)
        
    print(f"\n✅ Successfully saved average activations to {AVG_ACTIVATION_PKL_PATH}")

except FileNotFoundError:
    print(f"❌ Activation file not found. Did you run the experiment in Cell 8?")
except Exception as e:
    print(f"An error occurred: {e}")

### 💡 How to Use the Saved Averages

In any future notebook or script, you can now easily load and use these averaged tensors.

In [ ]:
import pickle

AVG_PATH = "./results/last_fold_average_activations.pkl"

try:
    with open(AVG_PATH, "rb") as f:
        avg_activations = pickle.load(f)

    print("Loaded average activations!")
    print(f"Available layers: {list(avg_activations.keys())}")

    # Example: Get the average activation for the 'dense_1' layer
    avg_dense_1 = avg_activations['dense_1']
    print(f"\nAverage 'dense_1' shape: {avg_dense_1.shape}")

except FileNotFoundError:
    print(f"File not found: {AVG_PATH}")